"""
理论来源：
1. https://xueqiu.com/1795556147/150797873
2. all weather strategy

"""

In [1]:
import pandas as pd
from datetime import timedelta
from tqdm.auto import tqdm
import numpy as np
import tushare as ts
from backtest_utils import *
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import copy
tqdm.pandas()
ts.set_token('6ddb307225c1b26eaabecc4e76561c9305d5bcad10bb6b882a3e63df')
pro = ts.pro_api()
start_dt = '20191201'
end_dt = '20240701'
test_dt = '20200701'

In [2]:
data = pd.DataFrame()
stock_list = ['510300.SH','510500.SH','513500.SH','513660.SH','511010.SH','511260.SH','518880.SH','510170.SH','159985.SZ']
stocks = stock_list
for stock in stock_list:
    df = pro.fund_daily(ts_code=stock, start_date=start_dt, end_date=end_dt)
    data  = pd.concat([data, df], ignore_index=True)

data.to_csv("etf.csv")
data = pd.read_csv('etf.csv')
data = data.sort_values(by = ['ts_code','trade_date'],ascending = True)
data['pct_chg']=data['pct_chg']/100
original_data = data

In [3]:
trade_cal_SSE = pro.trade_cal(exchange='SSE', start_date=test_dt, end_date=end_dt, is_open=1)
trade_date = trade_cal_SSE['cal_date']
trade_date = trade_date[::-1]
trade_date.to_csv('trade_dates.csv', index=False)
trade_date = pd.read_csv('trade_dates.csv')
trade_date = trade_date['cal_date'].astype(str)
trade_date

0      20200701
1      20200702
2      20200703
3      20200706
4      20200707
         ...   
966    20240625
967    20240626
968    20240627
969    20240628
970    20240701
Name: cal_date, Length: 971, dtype: object

In [4]:
data

,Unnamed: 0,ts_code,trade_date,pre_close,open,high,low,close,change,pct_chg,vol,amount
9983,9983,159985.SZ,20191205,0.983,0.983,0.995,0.978,0.986,0.003,0.003052,122256.57,12005.606
9982,9982,159985.SZ,20191206,0.986,0.993,0.996,0.982,0.984,-0.002,-0.002028,281011.46,27811.112
9981,9981,159985.SZ,20191209,0.984,0.986,0.993,0.986,0.988,0.004,0.004065,110016.32,10891.210
9980,9980,159985.SZ,20191210,0.988,0.987,0.990,0.984,0.987,-0.001,-0.001012,53524.05,5283.263
9979,9979,159985.SZ,20191211,0.987,0.991,0.995,0.989,0.992,0.005,0.005066,135801.31,13472.875
...,...,...,...,...,...,...,...,...,...,...,...,...
6663,6663,518880.SH,20240625,5.290,5.300,5.303,5.292,5.302,0.012,0.002268,2849134.00,1509273.578
6662,6662,518880.SH,20240626,5.302,5.287,5.287,5.253,5.271,-0.031,-0.005847,3337006.00,1758334.548
6661,6661,518880.SH,20240627,5.271,5.236,5.245,5.231,5.242,-0.029,-0.005502,2674882.00,1401342.548
6660,6660,518880.SH,20240628,5.242,5.283,5.299,5.277,5.297,0.055,0.010492,2758656.00,1458430.077


In [5]:
#加杠杆
data,leverage = calculate_vol(data,0.004,trade_date)

TypeError: calculate_vol() missing 1 required positional argument: 'vol_day'

In [ ]:
print(leverage)

In [ ]:

stock_list = ['510300.SH','510500.SH','513500.SH','513660.SH','511010.SH','511260.SH','518880.SH','510170.SH','159985.SZ']
stocks = stock_list
_, _, date_stocks = supporting_data(data, trade_date, stocks)
stocks_dict = {
    'stocks_1': ['518880.SH', '510170.SH','159985.SZ'],
    'stocks_2': ['510300.SH', '510500.SH', '513500.SH', '513660.SH', '510170.SH'],
    'stocks_3': ['518880.SH','159985.SZ', '511260.SH', '511010.SH'],
    'stocks_4': ['510300.SH', '510500.SH', '513500.SH', '513660.SH', '511260.SH', '511010.SH']
}

#test 数据，结果应与fixed一样
"""
stock_list = ['510300.SH', '510500.SH', '513500.SH', '513100.SH', '511260.SH', '511010.SH', '510170.SH','518880.SH','510410.SH']
stocks = stock_list
_, _, date_stocks = supporting_data(data, trade_date, stocks)
stocks_dict = {
    'stocks_1': ['510300.SH', '510500.SH', '513500.SH', '513100.SH', '511260.SH', '511010.SH', '510170.SH','518880.SH','510410.SH'],
    'stocks_2': ['510300.SH', '510500.SH', '513500.SH', '513100.SH', '511260.SH', '511010.SH', '510170.SH','518880.SH','510410.SH'],
    'stocks_3': ['510300.SH', '510500.SH', '513500.SH', '513100.SH', '511260.SH', '511010.SH', '510170.SH','518880.SH','510410.SH'],
    'stocks_4': ['510300.SH', '510500.SH', '513500.SH', '513100.SH', '511260.SH', '511010.SH', '510170.SH','518880.SH','510410.SH']
}

"""

results = four_quarters(data,trade_date,stocks_dict)


In [ ]:
date_stocks

In [ ]:
four_quarters = {}
for keys,values in results.items():
    for stock,weights in values.items():
        if stock in four_quarters.keys():
            four_quarters[stock]+= 1/4 * np.array(weights)
        else:
            four_quarters[stock] = 1/4 * np.array(weights)
            
# 确保顺序一致
code_seq = list(date_stocks.values())[0]
sorted_data = {code: four_quarters[code] for code in code_seq}
four_quarters = sorted_data

In [ ]:
four_quarters

In [ ]:
#计算 date_weights
date_weights =  {date:[] for date in trade_date}
for i,date in enumerate(trade_date):
    for stock,weights in four_quarters.items():
        date_weights[date].append(weights[i])


In [ ]:
date_weights

In [ ]:
date_stocks

In [ ]:
#初始资金、交易费率、偏离换仓参数
#注：原收益率未除100，calculate_daily_returns除以了100
daily_returns,net_values,position_change,holdings_diffs,stock_daily_returns = calculate_daily_returns(data,date_stocks,date_weights,1e6,0.00015,0.06,leverage,bond_rate)

In [ ]:
trade_date = pd.to_datetime(trade_date, format='%Y%m%d')
plt.plot(trade_date,holdings_diffs,label=stocks)
plt.title('Holdings_difference')
plt.xlabel('Days')
plt.ylabel('Deviation')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(14, 7))
for stock, weights in four_quarters.items():
    plt.plot(trade_date, weights, label=stock)
plt.legend(title='Stock')

for change_date in position_change:
    change_date = pd.to_datetime(change_date, format = '%Y%m%d')
    plt.axvline(change_date, color='red', linestyle='--', linewidth=1)
# 添加标题和轴标签
plt.title('Daily Stock Holding Proportions')
plt.xlabel('Date')
plt.ylabel('Holding Proportion')
plt.grid(True)

plt.savefig('stock_holdings_proportions_5.png', format='png', dpi=300)
# 显示图表
plt.show()

In [ ]:
plot_cumulative_returns(daily_returns,trade_date)

In [ ]:
win_rate, sharpe_ratio, max_drawdown = calculate_performance_metrics(daily_returns)

print(f"Win Rate: {win_rate}")
print(f"Sharpe Ratio: {sharpe_ratio}")
print(f"Max Drawdown: {max_drawdown}")

In [ ]:
data =pd.read_csv('etf.csv')
plot_etf_returns(data,test_dt)